In [ ]:
from openai import OpenAI

def generate_test_code(
    fqn: str,
    signature: str,
    jimple: str,
    callees: str,
    method_modifiers: str,
    class_modifiers: str,
    method_javadoc: str,
    method_body: str,
    imports: str,
    class_javadoc: str,
    class_fnc: str
) -> str:
    """
    Generates a complete and verbose Java unit test using OpenAI based on detailed information about a focal method.

    Returns:
        str: The generated Java test code as a string.
    """

    client = OpenAI(api_key="not-telling-you")

    prompt = f"""
**Test Generator {{

@persona {{
You are a Java expert and a professional test engineer.
Your role is to write high-coverage, verbose, and well-structured JUnit 4 test code for a **single Java method** under test.
Your output must consist of code only — NO explanation or commentary. Output a complete and compilable Java test file from package declaration to the final closing brace.
}}

@terminology {{
- focal_method_fqn: The fully qualified name of the class that contains the method under test.
- focal_method_signature: The declaration/signature of the method to be tested.
- focal_method_jimple: The Jimple representation of the focal method, showing low-level control and data flow.
- focal_method_body: The method's original Java source code.
- focal_method_modifiers: Access and other modifiers on the focal method (if present).
- focal_class_modifiers: Access and other modifiers on the focal class (if present).
- focal_method_javadoc: JavaDoc comment above the method (if present).
- focal_class_javadoc: JavaDoc comment above the class (if present).
- callee_edges: All edges to other methods that the focal method calls, including their signatures.
- field_and_constructor_declarations: Contains a single continuous string that includes all field declarations (modifer (useful), type, name)\
    —each accompanied by its JavaDoc comment and separated by semicolons—followed by the constructors of the focal class, all concatenated in the same string.
- import_statements: All necessary import statements found in the focal class file (excluding test imports).
}}

@instruction {{

@command: Using ALL available information, generate a verbose and high-coverage unit test for the provided focal method using JUnit 4 and JDK 8.
The test should aim to exercise as many execution paths as possible, leveraging the Jimple code, class/method modifiers, method calls, JavaDoc, and class context (fields & constructors).
If exception or edge-case paths are possible per Jimple/control flow, include `try-catch` blocks.
If the method or containing class is abstract, provide a concrete implementation as required.
Always consider edge cases: e.g., nulls, empty strings/arrays/collections, boundary values, and large or unusual inputs.
Make use of JavaDoc (when available) to reason about preconditions, expectations, and side effects.

@rule1: Output only the Java test code — no explanation, no markdown formatting, no commentary.
@rule2: Use JUnit 4 only. All test methods must at least include `@Test` and use `org.junit.Test` and `static org.junit.Assert.*` at the minimum.
@rule3: If the focal_method_body has many if-else branches, create a separate test method/case for each branch to ensure high coverage.
@rule4: Also include any additional import statements from the focal class provided in import_statements.
@rule5: Do not include or modify the focal method source in the test file.
@rule6: If the focal method is private or within a private/inner class, use Java Reflection to access and test it.
@rule7: If the focal method is accessing private/protected class fields, use Java Reflection to access and test them. \
    Relevant class fields modifiers can be found in the field_and_constructor_declarations.
@rule8: If the focal method throws exceptions, wrap calls in try-catch blocks. Do not use assertThrows, to maintain compatibility with JUnit 4.
@rule9: Name the test class in this format: <ClassName>_<MethodName>_<MethodParameterTypes>_Test.
@rule10: The test class must be a valid standalone Java file — include package declaration provided in the focal_method_fqn.
@rule11: When instantiating the focal class or its dependencies, derive constructor and field usage from field_and_constructor_declarations and callee_edges where possible.
@rule12: If some field, constructor, or callee information is missing, adapt gracefully to what is available.
@rule13: Each test method must cover a unique logical path or edge case. Prefer more tests with a variety of inputs over fewer.
}}

@format {{

@input:
###focal_method_fqn
{fqn}

###focal_method_signature
{signature}

###focal_method_jimple
{jimple}

###focal_method_body
{method_body}

###focal_method_modifiers
{method_modifiers}

###focal_class_modifiers
{class_modifiers}

###focal_method_javadoc
{method_javadoc}

###focal_class_javadoc
{class_javadoc}

###callee_edges
{callees}

###field_and_constructor_declarations
{class_fnc}

###import_statements
{imports}

@output:
###generated_test_code
}}
}}"""

    response = client.chat.completions.create(
        # Use new OpenAI gpt 4.1
        model= "gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=16000
    )

    return response.choices[0].message.content

In [ ]:
import os
import re
import pandas as pd

# Load CSV
df = pd.read_csv("/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task4-1.csv")

# Define the test base path
base_dir = "/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy"

def clean_code(code):
    code = re.sub(r'^```java\s*|```$', '', code.strip(), flags=re.MULTILINE)
    return code

for index, row in df.iterrows():
    # === 1. Extract inputs
    fqn = row['FQN']
    signature = row['Signature']
    jimple = row['Jimple']
    callees = row['Callees']
    method_modifiers = row['MethodModifiers']
    class_modifiers = row['ClassModifiers']
    method_javadoc = row['JavaDoc']
    method_body = row['MethodBody']
    imports = row['Imports']
    class_javadoc = row['classJavaDoc']
    class_fnc = row['classFnCs']

    # === 2. Generate test code
    raw_test_code = generate_test_code(
        fqn=fqn,
        signature=signature,
        jimple=jimple,
        callees=callees,
        method_modifiers=method_modifiers,
        class_modifiers=class_modifiers,
        method_javadoc=method_javadoc,
        method_body=method_body,
        imports=imports,
        class_javadoc=class_javadoc,
        class_fnc=class_fnc
    )

    # === 3. Clean the code
    formatted_code = clean_code(raw_test_code)
    df.at[index, "Generated Code"] = raw_test_code
    df.at[index, "Code After Formatting"] = formatted_code

    # === 4. Extract class and package name
    package_match = re.search(r'package\s+([\w\.]+);', formatted_code)
    if not package_match:
        print(f"[ERROR] Package not found in row {index}")
        continue

    package_path = package_match.group(1).replace('.', '/')
    relative_dir = os.path.join("src/test/java", package_path)

    class_match = re.search(r'public\s+class\s+(\w+)|class\s+(\w+)', formatted_code)
    if not class_match:
        print(f"[ERROR] Class name not found in row {index}")
        continue

    class_name = class_match.group(1) or class_match.group(2)
    file_name = f"{class_name}.java"
    save_path = os.path.join(base_dir, relative_dir, file_name)

    # === 5. Record the save path
    relative_path = os.path.relpath(save_path, start=base_dir)
    df.at[index, "Saved Path"] = relative_path

# Save updated dataframe
df.to_csv("/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task4-2.csv", index=False)
